In [2]:
import numpy as np
import utils 
import pandas as pd
import os
import matplotlib.pyplot as plt
import librosa
import utils
from PIL import Image
from tqdm import tqdm


## Attempt to reduce data dimensionality with Structural Similarity Index (SSIM)
The entire code is in the file utils.py, it was not reported here because it is refered to an attempt that didn't brought to any result we wanted to keep.
The idea was to remove all the images with an SSIM of 0.95 or higher from the classes with more than 500 elements.
The results were not as expected, so we decided to keep all the images.

In [2]:
utils.create_csv_for_folders('Spettrogrammi/Target/', 8)

In [3]:
utils.create_csv_for_folders('Spettrogrammi/Non-Target/', 8)

In [ ]:
utils.merge_csv_files('Spettrogrammi/Target/dati ssim/SSIM', 'Spettrogrammi/Target/ssim_results.csv')

In [ ]:
utils.merge_csv_files('Spettrogrammi/Non-Target/dati ssim/SSIM', 'Spettrogrammi/Non-Target/ssim_results.csv')

In [ ]:
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/ssim_results.csv')
pd.set_option('display.float_format', '{:.3f}'.format)
df_non_target.describe()

In [ ]:
df_target = pd.read_csv('Spettrogrammi/Target/ssim_results.csv')
pd.set_option('display.float_format', '{:.3f}'.format)
df_target.describe()

In [ ]:
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/df_paths.csv')
df_target = pd.read_csv('Spettrogrammi/Target/df_paths.csv')

df_non_target.to_csv('final_dataset/df_paths_non_target.csv', index=False)
df_target.to_csv('final_dataset/df_paths_target.csv', index=False)

# Split Dataset in Train, validation e Test

### Extracted the numbers of audios for each classes in order to avoid splitting the same audio in the different sets and removed the classes with less than 8 audios.

The results of the operations slightly changes if executed on MacOS or Windows because of how different OS handle the files.

In [2]:
df_target = pd.read_csv('Spettrogrammi/Target/df_paths.csv')
df_non_target = pd.read_csv('Spettrogrammi/Non-Target/df_paths.csv')

In [3]:
utils.estrai_info_da_csv('Spettrogrammi/Target/df_paths.csv', 'Spettrogrammi/Target/df_class_num.csv')
utils.estrai_info_da_csv('Spettrogrammi/Non-Target/df_paths.csv', 'Spettrogrammi/Non-Target/df_class_num.csv')

In [4]:
utils.filter_csv('Spettrogrammi/Target/df_class_num.csv', 'Spettrogrammi/Target/df_paths.csv', 'Spettrogrammi/Target/df_paths_filtered.csv')
utils.filter_csv('Spettrogrammi/Non-Target/df_class_num.csv', 'Spettrogrammi/Non-Target/df_paths.csv', 'Spettrogrammi/Non-Target/df_paths_filtered.csv')

In [5]:
def split_files_into_sets(input_csv_path, train_csv_path, val_csv_path, test_csv_path):
    # Imposta il seed per il generatore di numeri casuali
    np.random.seed(0)

    # Carica il file CSV in un DataFrame
    df = pd.read_csv(input_csv_path)

    # Crea una nuova colonna per il nome del file senza la parte "_resampled"
    df['File'] = df['FilePath'].apply(lambda x: x.split('_resampled')[0])

    # Inizializza una nuova colonna per il set
    df['Set'] = ''

    # Raggruppa per classe
    grouped = df.groupby('Classe')

    # Per ogni classe, assegna un set a ciascun file unico
    for name, group in grouped:
        # Ottieni un elenco di file unici
        unique_files = group['File'].unique()

        # Calcola le dimensioni dei set
        train_size = int(0.8 * len(unique_files))
        val_size = int(0.1 * len(unique_files))
        test_size = len(unique_files) - train_size - val_size

        # Assegna un set a ciascun file unico
        file_sets = ['train'] * train_size + ['val'] * val_size + ['test'] * test_size

        # Mescola l'elenco dei set di file
        np.random.shuffle(file_sets)

        # Crea un dizionario che mappa ciascun file al suo set
        file_to_set = dict(zip(unique_files, file_sets))

        # Assegna ogni riga al set del suo file
        df.loc[group.index, 'Set'] = group['File'].apply(lambda x: file_to_set[x])

    # Crea tre DataFrame separati per i set di addestramento, validazione e test
    df_train = df[df['Set'] == 'train']
    df_val = df[df['Set'] == 'val']
    df_test = df[df['Set'] == 'test']

    # Crea le cartelle di destinazione se non esistono già
    os.makedirs(os.path.dirname(train_csv_path), exist_ok=True)
    os.makedirs(os.path.dirname(val_csv_path), exist_ok=True)
    os.makedirs(os.path.dirname(test_csv_path), exist_ok=True)

    # Salva i DataFrame separati come nuovi CSV
    df_train.to_csv(train_csv_path, index=False)
    df_val.to_csv(val_csv_path, index=False)
    df_test.to_csv(test_csv_path, index=False)

In [33]:
split_files_into_sets('Spettrogrammi/Target/df_paths_filtered.csv', 'final_dataset/training/df_paths_target_train.csv', 'final_dataset/validation/df_paths_target_val.csv', 'final_dataset/test/df_paths_target_test.csv')

In [34]:
split_files_into_sets('Spettrogrammi/Non-Target/df_paths_filtered.csv', 'final_dataset/training/df_paths_non_target_train.csv', 'final_dataset/validation/df_paths_non_target_val.csv', 'final_dataset/test/df_paths_non_target_test.csv')

### Distribuzione dei file nel training set

In [2]:
pd.set_option('display.max_rows', None)
df_train_target = pd.read_csv('final_dataset/training/df_paths_target_train.csv')
max_target = np.max(df_train_target['Classe'].value_counts())
max_target

12339

In [3]:
pd.set_option('display.max_rows', None)
df_train_non_target = pd.read_csv('final_dataset/training/df_paths_non_target_train.csv')
max_non_target = np.max(df_train_non_target['Classe'].value_counts())
max_non_target

1822

In [4]:
df_train_target['Classe'].value_counts()

Classe
Tanker           12339
Passengership    11258
Tug              10727
Cargo            10169
Vessel             170
Name: count, dtype: int64

In [5]:
df_train_non_target['Classe'].value_counts()

Classe
Sperm Whale                           1822
Humpback Whale                        1657
Fin, Finback Whale                     818
Soundscape                             239
Fraser_s Dolphin                       224
Long-Finned Pilot Whale                191
Short-Finned (Pacific) Pilot Whale     177
Grampus, Risso_s Dolphin               139
Killer Whale                           133
Spinner Dolphin                        129
Striped Dolphin                        128
Common Dolphin                         111
Bowhead Whale                          107
Melon Headed Whale                      85
Walrus                                  77
Bearded Seal                            75
Atlantic Spotted Dolphin                71
White-sided Dolphin                     70
Harp Seal                               69
Ross Seal                               69
Clymene Dolphin                         68
Pantropical Spotted Dolphin             67
Beluga, White Whale                     65
Nort

## Data Augmentation

In [6]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

def load_spectrogram(image_path):
    image = Image.open(image_path)
    spectrogram = np.array(image) / 255 # Normalizza i valori a 0-1
    return spectrogram, image.mode

def time_shift_spectrogram(image_path, output_path, i, duration = 3):
    """
    Perform a time shift on a spectrogram image.

    Parameters:
    - image_path (str): Path to the input spectrogram image.
    - duration (int): Duration of the audio in seconds.
    - shift_seconds (int): Number of seconds to shift.
    - output_path (str): Path to save the shifted spectrogram image.
    """
    # Load the image
    image = Image.open(image_path)
    
    # Convert the image to a NumPy array, preserving all four channels (RGBA)
    image_array = np.array(image, dtype=np.uint8)
    
    shift_seconds = [0.08, 0.16, 0.24, 0.32, 0.40, 0.48, 0.56, 0.64, 0.72, 
                     0.80, 0.88, 0.96, 1.04, 1.12, 1.20, 1.28, 1.36, 1.44, 1.52]
    
    # Calculate the number of pixels corresponding to the shift_seconds
    width = image_array.shape[1]
    pixels_per_second = width // duration
    shift_pixels = int(pixels_per_second * shift_seconds[i])
    
    # Create a new array to hold the time-shifted spectrogram
    shifted_image_array = np.zeros_like(image_array, dtype=np.uint8)
    
    # Shift the image by the calculated number of pixels to the right
    shifted_image_array[:, shift_pixels:] = image_array[:, :-shift_pixels]
    
    # Create a "silence" array for the shifted part filled with black pixels
    silence = np.zeros((image_array.shape[0], shift_pixels, image_array.shape[2]), dtype=np.uint8)
    silence[:, :, 3] = 255  # Set the alpha channel to 255 for full opacity
    
    # Apply the silence to the beginning of the shifted image
    shifted_image_array[:, :shift_pixels] = silence
    
    # Convert the shifted array back to an image
    shifted_image = Image.fromarray(shifted_image_array)
    
    # Save the shifted image
    shifted_image.save(output_path)
    
    
def add_white_noise(spectrogram, mode, save_path, i):
    noise_factor = [0.0011, 0.0012, 0.0013, 0.0014, 0.0015, 0.0016, 0.0017, 0.0018, 0.0019, 0.0020, 
                    0.0021, 0.0022, 0.0023, 0.0024, 0.0025, 0.0026, 0.0027, 0.0028, 0.0029]
    
    # for noise_factor in noise_factor:
    noise = np.random.randn(*spectrogram.shape) * noise_factor[i] # Modificare sta riga se necessario per avere più opzioni di rumore (ad esempio fare /2, ecc....)
    augmented_spectrogram = spectrogram + noise
    np.clip(augmented_spectrogram, 0, 1)  # Clippa i valori tra 0 e 1
    
    augmented_spectrogram = (augmented_spectrogram * 255).astype(np.uint8)  # Converti i valori a 0-255
    image = Image.fromarray(augmented_spectrogram, mode)  # Converti in immagine mantenendo il mode originale
    image.save(save_path)

# def augment_spectrogram(image_path, save_dir, width_spectogram=930, shift_seconds=1.0):
#     
#     # Carica lo spettrogramma originale
#     spectrogram, mode = load_spectrogram(image_path)
# 
#     # Calcola la quantità di shift in pixel
#     shift_pixels = int(shift_seconds * width_spectogram/3)
# 
#     # Applica il time-shift
#     # shifted_spectrogram = time_shift_spectrogram(spectrogram, shift_pixels)
#     shifted_image_path = os.path.join(save_dir, f'shifted_{os.path.basename(image_path)}')
#     time_shift_spectrogram(image_path, shifted_image_path)
#     
# 
#     # Crea la directory se non esiste
#     os.makedirs(save_dir, exist_ok=True)
# 
#     # Salva lo spettrogramma shiftato
#     # save_spectrogram(shifted_spectrogram, shifted_image_path, mode)
# 
#     # Applica il white noise
#     noisy_image_path = os.path.join(save_dir, f'noisy_{os.path.basename(image_path)}')
#     add_white_noise(spectrogram, mode, noisy_image_path)
# 
# 
#     # Salva lo spettrogramma con il rumore
#     # save_spectrogram(noisy_spectrogram, noisy_image_path, mode)
# 
#     return shifted_image_path, noisy_image_path

def time_shift(image_path, save_dir, i):
    # shift_seconds = [1.0, 0.5, 0.75, 0.25][np.random.randint(1)]
    # Carica lo spettrogramma originale
    # spectrogram, mode = load_spectrogram(image_path)

    # Calcola la quantità di shift in pixel
    # shift_pixels = int(shift_seconds * width_spectogram / 3)

    # Applica il time-shift
    # shifted_spectrogram = time_shift_spectrogram(spectrogram, shift_pixels)
    shifted_image_path = os.path.join(save_dir, f'shifted_{i}_{os.path.basename(image_path)}')
    
    if not os.path.exists(shifted_image_path):
        time_shift_spectrogram(image_path, shifted_image_path, i)
        os.makedirs(save_dir, exist_ok=True)
        return shifted_image_path, True
    else:
        return "",False
    # Salva lo spettrogramma shiftato
    # save_spectrogram(shifted_spectrogram, shifted_image_path, mode)


def white_noise(image_path, save_dir, i):
    # Carica lo spettrogramma originale
    spectrogram, mode = load_spectrogram(image_path)
    
    # Applica il white noise
    noisy_image_path = os.path.join(save_dir, f'noisy_{i}_{os.path.basename(image_path)}')
    
    if not os.path.exists(noisy_image_path):
        add_white_noise(spectrogram, mode, noisy_image_path, i)
        os.makedirs(save_dir, exist_ok=True)
        return True
    else:
        return False
    


def increase_and_decrease_volume(spectrogram, save_path, mode, i, type):
    if type == 'i':
        volume_factor = [1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45, 1.5, 
                         1.55, 1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1.9, 1.95, 2]
    else: # type == 'd'
        volume_factor = [0.15, 0.175, 0.19, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 
                         0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
        
    # Aumenta il volume solo sui canali RGB, lasciando il canale alpha invariato
    spectrogram[..., :3] *= volume_factor[i]

    # Clippa i valori tra 0 e 255
    spectrogram = np.clip(spectrogram, 0, 255)

    # Salva lo spettrogramma modificato
    amplified_image = Image.fromarray(spectrogram.astype(np.uint8), mode)
    amplified_image.save(save_path)
    
def increase_and_decrease(image_path, save_dir, i, type):
    image = Image.open(image_path)
    spectrogram = np.array(image).astype(np.float32)
    mode = image.mode
    
    if type == 'i':
        increase_image_path = os.path.join(save_dir, f'increase_{i}_{os.path.basename(image_path)}')
    else: # type == 'd'
        increase_image_path = os.path.join(save_dir, f'decrease_{i}_{os.path.basename(image_path)}')
    
    if os.path.exists(increase_image_path):
        return False
    else:
        increase_and_decrease_volume(spectrogram, increase_image_path, mode, i, type)
        
        os.makedirs(save_dir, exist_ok=True)
        return True

def process_csv(csv_path, save_dir, value_counts):
    df = pd.read_csv(csv_path)
    max = np.max(value_counts)
    i = 0

    while np.any(value_counts < max):
        for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing images"):
            image_path = row.iloc[0]

            parts = image_path.split('/')
            relevant_parts = parts[1:-1]

            destination_dir = os.path.join(save_dir, *relevant_parts[:-1])
            destination_path = os.path.join(destination_dir, relevant_parts[-1])

            os.makedirs(destination_path, exist_ok=True)
            # If the curent class has reached the maximum, go to the next iteration
            if value_counts.loc[parts[2]] >= max:
                continue

            shifted_path, done_f = time_shift(image_path, destination_path, i)
            if done_f:
                value_counts.loc[parts[2]] += 1
            
            #Portion of the code to merge time shift and increase and decrease, done_f is a flag that indicates if the time shift has been done to then also do the increase and decrease, if there was no done_f in case of duplicate, the function would crash
            if value_counts.loc[parts[2]] < max and done_f:
                 done = increase_and_decrease(shifted_path, destination_path, i, 'i')
                 if done:
                    value_counts.loc[parts[2]] += 1
                 
            #if value_counts.loc[parts[2]] < max and done_f:
            #    done = increase_and_decrease(shifted_path, destination_path, i, 'd')
            #    if done:
            #        value_counts.loc[parts[2]] += 1
            #----------------
            if value_counts.loc[parts[2]] < max:
               done = white_noise(image_path, destination_path, i)
               if done:
                    value_counts.loc[parts[2]] += 1

            if value_counts.loc[parts[2]] < max:
                done = increase_and_decrease(image_path, destination_path, i, 'i')
                if done:
                    value_counts.loc[parts[2]] += 1

            if value_counts.loc[parts[2]] < max:
                done = increase_and_decrease(image_path, destination_path, i, 'd')
                if done:
                    value_counts.loc[parts[2]] += 1

            i += 1
            if i == 19:
                i = 0

In [7]:
process_csv('final_dataset/training/df_paths_target_train.csv', 'final_dataset/training', df_train_target['Classe'].value_counts())

Processing images: 100%|██████████| 44663/44663 [00:36<00:00, 1230.89it/s] 


- bilanciare
- pregare che funzioni

In [39]:
def add_rows_to_csv(root_dir, csv_path, new_csv_path):
    # Carica il CSV esistente in un DataFrame
    df = pd.read_csv(csv_path)

    # Scorri tutte le sottocartelle e i file .png
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith('.png'):
                # Crea il percorso completo del file
                full_path = os.path.join(dirpath, filename)

                # Estrai il nome della classe dalla sottocartella
                class_name = os.path.basename(dirpath)

                # Costruisci il nome del file rimuovendo la parte '_resampled'
                file_name = full_path.split('_resampled')[0]

                # Crea una nuova riga
                new_row = pd.DataFrame({'FilePath': [full_path], 'Classe': [class_name], 'File': [file_name], 'Set': ['train']})

                # Aggiungi la nuova riga al DataFrame
                df = pd.concat([df, new_row], ignore_index=True)

    # Salva il DataFrame modificato in un nuovo CSV
    df.to_csv(new_csv_path, index=False)

In [40]:
add_rows_to_csv('final_dataset/training/Target', 'final_dataset/training/df_paths_target_train.csv', 'final_dataset/training/new_df_paths_target_train.csv')

In [41]:
add_rows_to_csv('final_dataset/training/Non-Target', 'final_dataset/training/df_paths_non_target_train.csv', 'final_dataset/training/new_df_paths_non_target_train.csv')

In [3]:
def merge_csv_files(csv1_path, csv2_path, output_path):
    # Carica i due CSV in due DataFrame separati
    df1 = pd.read_csv(csv1_path)
    df2 = pd.read_csv(csv2_path)

    # Unisci i due DataFrame
    df = pd.concat([df1, df2], ignore_index=True)

    # Crea la colonna "Label" in base alla presenza di "Non-Target" o "Target" in FilePath
    df['Label'] = df['FilePath'].apply(lambda x: 'Non-Target' if "Non-Target" in x else 'Target')

    # Mescola le righe del DataFrame
    df = df.sample(frac=1).reset_index(drop=True)

    # Salva il DataFrame modificato in un nuovo CSV
    df.to_csv(output_path, index=False)

In [71]:
merge_csv_files('final_dataset/training/new_df_paths_target_train.csv', 'final_dataset/training/new_df_paths_non_target_train.csv', 'final_dataset/training/df_paths_train.csv')

In [87]:
def process_csv_merge(csv_path, save_dir, value_counts):
    df = pd.read_csv(csv_path)
    max = np.max(value_counts)
    i = 0

    while np.any(value_counts < max):
        for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing images"):
            image_path = row.iloc[0]

            parts = image_path.split('/')
            if 'Spettrogrammi' in parts[0]:
                relevant_parts = parts[1:-1]
                label = parts[1]
            else:
                relevant_parts = parts[2:-1]
                label = parts[2]

            destination_dir = os.path.join(save_dir, *relevant_parts[:-1])
            destination_path = os.path.join(destination_dir, relevant_parts[-1])
            
            os.makedirs(destination_path, exist_ok=True)
            # If the curent class has reached the maximum, go to the next iteration
            if value_counts.loc[label] >= max:
                continue


            if value_counts.loc[label] < max and "shifted" in image_path and "increase" not in image_path:
                done = increase_and_decrease(image_path, destination_path, i, 'd')
                if done:
                    value_counts.loc[parts[2]] += 1


            i += 1
            if i == 19:
                i = 0

In [88]:
process_csv_merge('final_dataset/training/df_paths_train.csv', 'final_dataset/training/', df_train_target['Label'].value_counts())

Processing images: 100%|██████████| 121821/121821 [01:21<00:00, 1495.76it/s] 


In [96]:
add_rows_to_csv('final_dataset/training/Non-Target', 'final_dataset/training/df_paths_non_target_train.csv', 'final_dataset/training/new_df_paths_non_target_train.csv')

In [97]:
merge_csv_files('final_dataset/training/new_df_paths_target_train.csv', 'final_dataset/training/new_df_paths_non_target_train.csv', 'final_dataset/training/df_paths_train.csv')

In [98]:
pd.set_option('display.max_rows', None)
df_train_target = pd.read_csv('final_dataset/training/df_paths_train.csv')
# df_train_target
max_target = np.max(df_train_target['Label'].value_counts())
max_target

61695

In [99]:
df_train_target['Label'].value_counts()

Label
Non-Target    61695
Target        61695
Name: count, dtype: int64

In [4]:
# Unione file csv per il validation set
merge_csv_files('final_dataset/validation/df_paths_target_val.csv', 'final_dataset/validation/df_paths_non_target_val.csv', 'final_dataset/validation/df_paths_val.csv')

In [6]:
# conta elementi png in una cartella e sottocartelle
def count_files_in_folder(folder_path):
    count = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith('.png'):
                count += 1
    return count

count_files_in_folder('/Users/daniela/Library/CloudStorage/OneDrive-UniversitàdiSalerno/I Anno/Secondo Semestre/Fondamenti di Visione Artificiale e Biometria/Progetto/biometria/final_dataset/Non-Target')

54679

In [4]:
merge_csv_files('final_dataset/training/df_paths_target_train.csv', 'final_dataset/training/df_paths_non_target_train.csv', 'final_dataset/training/df_paths_train_clean.csv')